# **Pipeline : Manejo de Datos - Circulación**


Este proyecto detalla el proceso completo llevado a cabo por el equipo consultor PAIT#11 para desarrollar un pipeline que permite recrear cada uno de los pasos realizados en la construcción de una base de datos destinada al Departamento de Circulación. El objetivo principal es garantizar la trazabilidad, eficiencia y automatización en la recopilación, transformación y almacenamiento de datos, facilitando así la toma de decisiones basada en información precisa y actualizada.  

A lo largo de este pipeline, se abordan etapas clave como la extracción de datos de diversas fuentes, la limpieza y transformación de los mismos para asegurar su consistencia, y finalmente su carga en la estructura diseñada. Además, se consideran prácticas óptimas para el manejo de grandes volúmenes de datos y la integración de modelos predictivos que contribuyan a la planificación eficiente de envíos y ventas.  

Gracias al trabajo coordinado del equipo PAIT#11, este enfoque estructurado permitirá no solo replicar el proceso con facilidad sino también optimizar la gestión de la información y fortalecer la capacidad analítica del Departamento de Circulación.  

Hojas de Datos Utilizadas:


1.   **Circulación_2021.csv**
2.   **Circulación_2022.csv**
2.   **Circulación_2023.csv**
2.   **Circulación_2024.csv**
2.   **Circulación_2025.csv**



*Este material fue desarrollado por el Equipo PAIT#11 en conjunto con Jaime Guell (Investigador de INCAE).*


### **Combinación de Archivos CSV en un Único DataFrame**


El código utiliza la biblioteca `pandas` para cargar múltiples archivos CSV (`circulacion_2021.csv` a `circulacion_2025.csv`), combinándolos en un único `DataFrame` mediante la función `pd.concat()`. La opción `ignore_index=True` renumera los índices para evitar duplicados. El `DataFrame` resultante se guarda como `circulacion.csv` sin incluir el índice original. Finalmente, se imprime un mensaje confirmando la operación.

In [ ]:
import pandas as pd

# Lista con los nombres de los archivos
archivos = ["circulacion_2021.csv", "circulacion_2022.csv", "circulacion_2023.csv", "circulacion_2024.csv", "circulacion_2025.csv"]

# Cargar y combinar los CSV
merged_df = pd.concat([pd.read_csv(archivo) for archivo in archivos], ignore_index=True)

# Guardar el resultado
merged_df.to_csv("circulacion.csv", index=False)

print("Merge completado y guardado como 'Circulcion_Merge_2021_2025.csv'.")


### **Adición de Columna con el Día de la Semana en Español para Archivos CSV Grandes**

El código utiliza `pandas` para procesar grandes volúmenes de datos en bloques (`chunksize=100000`), evitando problemas de memoria al trabajar con el archivo `circulacion.csv`. Convierte la columna `fecha_edicion` al formato `datetime` y crea una nueva columna `dia_semana` en español, usando un diccionario para traducir los días de la semana. La columna se guarda en minúsculas. Los resultados se escriben incrementalmente en `circulacion_dia.csv` utilizando el modo `'a'` (append) y controlando el encabezado para evitar duplicaciones. Finalmente, imprime un mensaje confirmando el éxito de la operación.

In [ ]:
import pandas as pd

# Lista con los nombres de los archivos
archivos = ["circulacion.csv"]

# Diccionario para traducir días al español
dias_semana_es = {
    'Monday': 'lunes',
    'Tuesday': 'martes',
    'Wednesday': 'miércoles',
    'Thursday': 'jueves',
    'Friday': 'viernes',
    'Saturday': 'sábado',
    'Sunday': 'domingo'
}

# Nombre del archivo de salida
output_file = "circulacion_dia.csv"

# Borrar el archivo si existe para empezar limpio
open(output_file, 'w').close()

# Procesar cada archivo en bloques
for archivo in archivos:
    # Carga por bloques para manejar grandes volúmenes de datos
    for chunk in pd.read_csv(archivo, chunksize=100000):
        # Convertir 'fecha_edicion' a formato datetime
        chunk['fecha_edicion'] = pd.to_datetime(chunk['fecha_edicion'], format='%m/%d/%Y', errors='coerce')

        # Crear columna 'dia_semana' en español y en minúsculas
        chunk['dia_semana'] = chunk['fecha_edicion'].dt.day_name().map(dias_semana_es).str.lower()

        # Guardar en CSV de forma incremental
        chunk.to_csv(output_file, mode='a', index=False, header=not bool(open(output_file).read(1)))

print("Merge completado y guardado como 'circulacion.csv' con la columna 'dia_semana' en español y minúsculas.")


### **Extracción de Año, Mes y Día desde la Fecha en un Archivo CSV**

El código carga el archivo `circulacion_dia.csv` en un `DataFrame` de `pandas` y convierte la columna `fecha_edicion` a formato `datetime` para facilitar la manipulación de fechas. A partir de esta columna, crea tres nuevas columnas: `año`, `mes` y `dia`, extrayendo respectivamente el año, el mes y el día. El `DataFrame` modificado se guarda como `circulacion_fecha.csv` sin incluir el índice original. Finalmente, muestra las primeras filas del `DataFrame` para verificar el resultado.

In [ ]:
# Cargar el archivo CSV con las columnas ya filtradas
df = pd.read_csv("circulacion_dia.csv")

# Convertir la columna fecha_edicion a formato datetime
df["fecha_edicion"] = pd.to_datetime(df["fecha_edicion"])

# Crear nuevas columnas para año, mes y día
df["año"] = df["fecha_edicion"].dt.year
df["mes"] = df["fecha_edicion"].dt.month
df["dia"] = df["fecha_edicion"].dt.day

# Guardar el resultado en un nuevo archivo CSV
df.to_csv("circulacion_fecha.csv", index=False)

# Mostrar las primeras filas del DataFrame
print(df.head())

### **Reordenamiento Personalizado de Columnas en un Archivo CSV**

El código carga el archivo `circulacion_fecha.csv` en un `DataFrame` de `pandas` y reorganiza las columnas según un orden personalizado dividido en categorías lógicas:
1. **Variables Temporales:** `fecha_edicion`, `año`, `mes`, `dia`, `dia_semana`.
2. **Variables de Pedido y Transacción:** `idorder`, `idpublication`, `publicacion`.
3. **Variables Geográficas:** `idregional`, `regional`, `iddeptogeo`, `departamento`, etc.
4. **Variables de Canal y Segmento:** `idchannel`, `nomCanal`, `idsegmento`.
5. **Variables de Ruta y Logística:** `idroute`, `ruta`, etc.
6. **Variables de Clientes y Personal:** `ordinario`, `adicional`, `posterior`, etc.

El `DataFrame` reorganizado se guarda como `circulacion_ordenada.csv` sin el índice original. Finalmente, se imprime un mensaje confirmando el éxito de la operación.

In [ ]:
import pandas as pd

# Cargar el archivo CSV original
df = pd.read_csv("circulacion_fecha.csv")

# Definir el nuevo orden de las columnas
columnas_ordenadas = [
    # 1. Variables Temporales
    'fecha_edicion', 'año', 'mes', 'dia', 'dia_semana',

    # 2. Variables de Pedido y Transacción
    'idorder', 'idpublication', 'publicacion',

    # 3. Variables Geográficas
    'idregional', 'regional', 'iddeptogeo', 'departamento', 'idmupiogeo',
    'municipio', 'idsector', 'sector', 'direccion', 'zona',

    # 4. Variables de Canal y Segmento
    'idchannel', 'nomCanal', 'idsegmento',

    # 5. Variables de Ruta y Logística
    'idroute', 'ruta', 'idcustomer', 'codigo_sectorista', 'sectorista',
    'idsupervisor', 'supervisor',

    # 6. Variables de Clientes y Personal
    'ordinario', 'adicional', 'posterior',
    'promo', 'flete', 'envio_total', 'cobrable', 'devuelto', 'vendido'
]

# Reordenar las columnas
df = df[columnas_ordenadas]

# Guardar el DataFrame reordenado en un nuevo archivo
df.to_csv("circulacion_ordenada.csv", index=False)

print("Las columnas han sido reordenadas y guardadas como 'circulacion_ordenada.csv'.")


### **Extracción de Componentes de Fecha y Actualización de CSV**

El código carga el archivo `circulacion_ordenada.csv` en un `DataFrame` utilizando `pandas` y convierte la columna `fecha_edicion` al formato `datetime` para facilitar la manipulación de fechas. A partir de esta columna, genera tres nuevas columnas:
- **`año`**: Extrae el año de `fecha_edicion`.
- **`mes`**: Extrae el mes.
- **`dia`**: Extrae el día.

Luego, guarda el `DataFrame` actualizado en un archivo llamado `salida.csv` sin incluir el índice original. Finalmente, muestra las primeras filas del `DataFrame` para verificar la correcta ejecución del proceso.

In [ ]:
import pandas as pd

# Cargar el archivo CSV con las columnas ya filtradas
df = pd.read_csv("circulacion_ordenada.csv")

# Convertir la columna fecha_edicion a formato datetime
df["fecha_edicion"] = pd.to_datetime(df["fecha_edicion"])

# Crear nuevas columnas para año, mes y día
df["año"] = df["fecha_edicion"].dt.year
df["mes"] = df["fecha_edicion"].dt.month
df["dia"] = df["fecha_edicion"].dt.day

# Guardar el resultado en un nuevo archivo CSV
df.to_csv("salida.csv", index=False)

# Mostrar las primeras filas del DataFrame
print(df.head())

### **Eliminación de Columnas Específicas en un CSV**


El código carga el archivo `salida.csv` en un `DataFrame` utilizando `pandas` y elimina las columnas `fecha_edicion`, `idpublication` y `publicacion` mediante la función `drop(columns=[...])`. Posteriormente, guarda el `DataFrame` modificado en un nuevo archivo llamado `salida1.csv` sin incluir el índice original. Finalmente, imprime un mensaje confirmando que las columnas han sido eliminadas y el archivo guardado correctamente.

In [ ]:
import pandas as pd

# Cargar el archivo CSV
archivo = 'salida.csv'
df = pd.read_csv(archivo)

# Hacer el drop de las columnas especificadas
df = df.drop(columns=['fecha_edicion', 'idpublication', 'publicacion'])

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv('salida1.csv', index=False)

print("Las columnas 'fecha_edicion', 'idpublicacion' y 'publicacion' han sido eliminadas y guardadas en 'datos_sin_columnas.csv'.")


# **Merge : Promociones, Feriados y Hoja madre (salida1.csv)**




### Carga y Preparación de Datos desde Múltiples CSV en Pandas


El código utiliza la biblioteca `pandas` para cargar tres archivos CSV: `salida1.csv`, `promociones.csv` y `dias_festivos_guatemala_2021_2025.csv`, especificando la codificación `latin-1` para evitar problemas con caracteres especiales. A continuación, convierte las columnas de fechas (`fecha_edicion`, `fecha_inicio`, `fecha_fin`, y `Fecha`) al tipo `datetime` para garantizar un formato consistente y facilitar operaciones basadas en fechas.

Luego, obtiene los valores únicos de la columna `Tipo` de `hoja_2` y los almacena en la lista `unique_tipo`, que probablemente se utilizará para generar columnas adicionales en futuros procesos. También extrae las fechas únicas de `hoja_1` y las guarda en `unique_dates`, preparando los datos para análisis adicionales o uniones basadas en fechas.

In [ ]:
# Load required libraries
import pandas as pd

# Load csv files
hoja_1 = pd.read_csv('salida1.csv', encoding='latin-1')
hoja_2 = pd.read_csv('promociones.csv', encoding='latin-1')
dias_festivos = pd.read_csv('dias_festivos_guatemala_2021_2025.csv', encoding='latin-1')

# Convert date columns to uniform data type
hoja_1['fecha_edicion'] = pd.to_datetime(hoja_1['fecha_edicion'], format='%Y-%m-%d')

hoja_2['fecha_inicio'] = pd.to_datetime(hoja_2['fecha_inicio'], format='%Y-%m-%d')
hoja_2['fecha_fin'] = pd.to_datetime(hoja_2['fecha_fin'], format='%Y-%m-%d')

dias_festivos['Fecha'] = pd.to_datetime(dias_festivos['Fecha'], format='%Y-%m-%d')

# Get unique values for Tipo from hoja_2. These values will be used for
# generating additional columns in the output file
unique_tipo = list(set(hoja_2['Tipo']))

# Get unique dates from hoja 1
unique_dates = hoja_1['fecha_edicion'].unique()


Construyendo un DataFrame auxiliar para ayudar.

Primero, una columna de fechas únicas ubicada en `hoja_1`.
Segundo, una columna para 'Feriados', donde se establecerá un 1 si la fecha única coincide con una fecha en el DataFrame `dias_festivos`.
Tercero, se crearán columnas para cada elemento en `unique_tipo`, con valores de 0.

### Creación de un DataFrame Auxiliar con Indicadores de Feriados y Tipos


El código crea un `DataFrame` auxiliar (`helper_df`) a partir de las fechas únicas (`unique_dates`) obtenidas previamente, asignándolas a una columna llamada `date`. Luego, agrega una columna binaria llamada `Feriados` que indica si cada fecha corresponde a un día festivo, basándose en el `DataFrame` `dias_festivos`. Esta columna toma el valor `1` si la fecha está presente en `dias_festivos['Fecha']` y `0` en caso contrario.

Posteriormente, el código añade una columna para cada valor único de `Tipo` presente en `unique_tipo`, inicializando todas las celdas de estas columnas en `0`. Esto prepara el `DataFrame` para almacenar información específica relacionada con cada `Tipo` en futuros procesos de análisis o agregación.

In [ ]:
# Building helper data frame
helper_df = pd.DataFrame(
    {'date': unique_dates}
)

# Add 'Feriados' column to helper dataframe
helper_df['Feriados'] = helper_df['date'].isin(dias_festivos['Fecha']).astype(int)

# Add a column for each 'Tipo'
for tipo in unique_tipo:
    helper_df[tipo] = 0



Construyendo otro DataFrame intermedio basado en `hoja_2` para facilitar el trabajo.

- Agrupar por 'Tipo'.
- Obtener la fecha más baja de `fecha_inicio` y la fecha más alta de `fecha_fin` para períodos de tiempo contiguos por 'Tipo'.

### Colapso de Intervalos de Fechas por Tipo


El código combina intervalos de fechas superpuestos o adyacentes en el `DataFrame` `hoja_2`, agrupados por `Tipo`.

- `collapse_intervals(intervals)`:
  Ordena los intervalos por fecha de inicio y los fusiona si se superponen o son adyacentes.

- `collapse_group(type_group)`:
  Aplica `collapse_intervals` a cada grupo de `Tipo` y devuelve un `DataFrame` con los intervalos colapsados.

- Resultado:
  El `DataFrame` `hoja_2_collapsed` contiene intervalos optimizados y únicos por tipo.

In [ ]:
# Helper function to collapse intervals. Given a list of intervals (tuple of start, end),
# merge overlaping and adjacent intervals
def collapse_intervals(intervals):
    # Sort intervals by start date
    intervals.sort(key=lambda x: x[0])
    collapsed = []

    for current_interval in intervals:
        if not collapsed:
            collapsed.append(list(current_interval))
        else:
            # Check if current interval starts within or right after the last merged interval
            # If so, merge them.
            if current_interval[0] <= collapsed[-1][1] + pd.Timedelta(days=1):
                # Merge the intervals by extending the end date if necessaru
                collapsed[-1][1] = max(collapsed[-1][1], current_interval[1])
            else:
                collapsed.append(list(current_interval))

    return collapsed


# Function to merge intervals for each group of p_type
def collapse_group(type_group):
    # Get group name
    group_name = type_group.name
    # Create a list of (start_Data, end_date) tuples for the group
    intervals = list(zip(type_group['fecha_inicio'], type_group['fecha_fin']))
    collapsed = collapse_intervals(intervals)

    collapsed_df = pd.DataFrame({
        'Tipo': [group_name] * len(collapsed),
        'fecha_inicio': [interval[0] for interval in collapsed],
        'fecha_fin': [interval[1] for interval in collapsed]
    })

    return collapsed_df

hoja_2_collapsed = hoja_2.groupby('Tipo', group_keys=False)[['fecha_inicio', 'fecha_fin']].apply(collapse_group)

Hagamos algo mágico:

Ahora, para todas las fechas únicas en `helper_df`, se verificará si caen dentro del rango por 'Tipo' definido en `hoja_2_collapsed`, y se establecerá un 1 en la columna correspondiente de `helper_df`.  

### Marcado de Fechas dentro de Intervalos para Cada Tipo

El código recorre cada fila del `DataFrame` `hoja_2_collapsed`, que contiene columnas: `Tipo`, `fecha_inicio` y `fecha_fin`. Para cada intervalo de fechas:

1. Obtiene los valores:
   Extrae el tipo (`tipo_h`), la fecha de inicio (`start_date`) y la fecha de fin (`end_date`).

2. Crea una máscara booleana (`in_range`):
   Genera una condición booleana para identificar las filas en `helper_df` donde la columna `date` cae dentro del rango definido por `start_date` y `end_date`.

3. Asigna valores a `helper_df`:  
   Utiliza `.loc` para actualizar la columna correspondiente a `tipo_h` en `helper_df`, asignando `1` a las fechas que cumplen la condición y manteniendo `0` en las demás.


### Resultado:
El `DataFrame` `helper_df` se completa con indicadores binarios (`1` o `0`) para cada tipo, señalando si una fecha específica se encuentra dentro de los intervalos establecidos en `hoja_2_collapsed`. Esto permite identificar rápidamente la aplicabilidad de cada tipo en función de las fechas.

In [ ]:
for idx, row in hoja_2_collapsed.iterrows():
    tipo_h = row['Tipo']
    start_date = row['fecha_inicio']
    end_date = row['fecha_fin']

    # Boolean mask to check if date is within range
    in_range = (helper_df['date'] >= start_date) & (helper_df['date'] <= end_date)

    # Set corresponding column to 1 where the condition is met
    helper_df.loc[in_range, tipo_h] = 1

The helper_df is complete. This df has for every unique date that appeared in hoja_1:
* A column with the dates
* A column for 'Feriados': set to 1 if it matches a date in the dias_festivos
* Columns for each unique Tipo from hoja 2: with it's corresponding Tipo set to 1 if there is a match in the date ranges from hoja_2


Now, the helper_df will be joined to hoja_1 to produce the expected outcome

### Unificación de Datos mediante Merge

Renombra `date` a `fecha_edicion` en `helper_df` y une `hoja_1` con `helper_df` usando `merge` en modo `left`. Guarda el resultado como `hoja_1_enhanced.csv`, agregando las columnas adicionales sin perder filas originales.

In [ ]:
# Pre-work: rename date column from helper_df to fecha edicion, in order to hava a cleaner result after merging
helper_df = helper_df.rename(columns={'date': 'fecha_edicion'})

# Merge the data
hoja_1_enhanced = hoja_1.merge(helper_df, how='left', on='fecha_edicion')

# Save to output file
hoja_1_enhanced.to_csv('hoja_1_enhanced.csv', index=False)

# **Merge: Hoja Madre (hoja_1_enhanced.csv)**

Integración y Limpieza de Datos con Pandas: Uniendo Archivos CSV y Excel Basados en Fechas y Municipios

Este código carga dos archivos (un CSV y un Excel) en DataFrames usando `pandas`, normaliza los nombres de columnas y municipios (minúsculas y sin espacios), y asegura que las fechas estén en el mismo formato. Luego, transforma los datos de lluvia usando `melt()` para tener una estructura uniforme con las columnas `fecha_edicion`, `municipio` y `lluvia`.

Después, filtra las filas para mantener solo las coincidencias exactas de municipio y fecha entre los DataFrames. Realiza un `merge` tipo `left` para combinar los datos, rellenando los valores faltantes en la columna `lluvia` con 0. Finalmente, guarda el DataFrame resultante en un nuevo CSV llamado `datos_limpios_con_temp.csv`.

In [ ]:
import pandas as pd
import os

# Definir rutas de los archivos
ruta_hoja_madre = "hoja_1_enhanced.csv.csv"
ruta_hoja_temp = "Temp match.xlsx"
ruta_salida_csv = "datos_limpios_con_temp.csv"

# Crear el directorio de salida si no existe
ruta_directorio = os.path.dirname(ruta_salida_csv)
if not os.path.exists(ruta_directorio):
    os.makedirs(ruta_directorio)

# Cargar los datos
print("Cargando datos...")
hoja_madre = pd.read_csv(ruta_hoja_madre)
hoja_temp = pd.read_excel(ruta_hoja_temp)

# Normalizar nombres de columnas eliminando espacios y convirtiendo a minúsculas
print("Normalizando nombres de columnas y municipios...")
hoja_madre.columns = hoja_madre.columns.str.strip().str.lower()
hoja_temp.columns = hoja_temp.columns.str.strip().str.lower()

# Asegurar que la fecha tiene el mismo formato
print("Formateando fechas...")
hoja_madre["fecha_edicion"] = pd.to_datetime(hoja_madre["fecha_edicion"], errors='coerce')
hoja_temp["fecha_edicion"] = pd.to_datetime(hoja_temp["fecha_edicion"], errors='coerce')

# Transformar la hoja de lluvia para que cada municipio sea una fila con su valor de lluvia
print("Transformando hoja de lluvia con melt()...")
hoja_lluvia = hoja_temp.melt(id_vars=["fecha_edicion"], var_name="municipio", value_name="lluvia")

# Normalizar nombres de municipios eliminando espacios extra
if "municipio" in hoja_madre.columns and "municipio" in hoja_lluvia.columns:
    hoja_madre["municipio"] = hoja_madre["municipio"].str.strip().str.lower()
    hoja_lluvia["municipio"] = hoja_lluvia["municipio"].str.strip().str.lower()
else:
    print("Error: La columna 'municipio' no se encuentra en uno de los DataFrames.")
    print("Columnas en hoja_madre:", hoja_madre.columns.tolist())
    print("Columnas en hoja_lluvia:", hoja_lluvia.columns.tolist())
    exit()

# Verificación previa al merge
print("Ejemplo de municipios en hoja madre:", hoja_madre["municipio"].unique()[:10])
print("Ejemplo de municipios en hoja lluvia:", hoja_lluvia["municipio"].unique()[:10])

# Filtrar solo los municipios y fechas que realmente tienen coincidencia
print("Filtrando solo coincidencias exactas antes del merge...")
hoja_lluvia_filtrada = hoja_lluvia[
    hoja_lluvia["municipio"].isin(hoja_madre["municipio"].unique()) &
    hoja_lluvia["fecha_edicion"].isin(hoja_madre["fecha_edicion"].unique())
]

# Realizar el merge asegurando coincidencias exactas de municipio y fecha
print("Realizando merge de datos con coincidencia exacta de fecha y municipio...")
hoja_madre = hoja_madre.merge(hoja_lluvia_filtrada, on=["fecha_edicion", "municipio"], how="left")

# Asignar 0 a los valores sin coincidencia
hoja_madre["lluvia"] = hoja_madre["lluvia"].fillna(0)

# Verificar valores después del merge
print("Ejemplo de datos después del merge:")
print(hoja_madre[["fecha_edicion", "municipio", "lluvia"]].head(10))

# Guardar en un único archivo CSV sin restricciones de tamaño
print("Guardando CSV corregido...")
hoja_madre.to_csv(ruta_salida_csv, index=False)

print(f"CSV guardado exitosamente en {ruta_salida_csv}")

Exportación y Muestreo de Datos con Pandas en Python

Este código carga un archivo CSV llamado `datos_limpios_con_temp.csv` en un DataFrame y normaliza los nombres de columnas a minúsculas y sin espacios, asegurando también que la columna `fecha_edicion` tenga el formato datetime correcto. Luego, toma una muestra aleatoria de 2000 filas y la guarda en dos formatos diferentes: CSV (`datos_muestra_2000.csv`) y Excel (`datos_muestra_2000.xlsx`). Además, exporta el DataFrame completo a un archivo CSV (`datos_limpios_con_temp_completo.csv`).

Para manejar grandes volúmenes de datos en Excel, el código utiliza `pd.ExcelWriter` y divide el DataFrame en varias hojas si supera el límite de 1,000,000 filas, guardando el resultado en `datos_limpios_con_temp.xlsx`. Finalmente, muestra un ejemplo de las primeras filas de la muestra aleatoria para verificar el contenido exportado.

In [ ]:
import pandas as pd

# Cargar los datos completos desde la nueva hoja de datos
hoja_madre = pd.read_csv("/content/datos_limpios_con_temp.csv")

# Normalizar nombres de columnas eliminando espacios y convirtiendo a minúsculas
hoja_madre.columns = hoja_madre.columns.str.strip().str.lower()

# Asegurar que la fecha tiene el mismo formato
hoja_madre["fecha_edicion"] = pd.to_datetime(hoja_madre["fecha_edicion"], errors='coerce')

# Tomar una muestra aleatoria de 2000 observaciones
hoja_muestra = hoja_madre.sample(n=2000, random_state=42)

# Guardar la muestra en un archivo CSV
hoja_muestra.to_csv("/content/datos_muestra_2000.csv", index=False)

# Guardar la muestra en un archivo Excel
hoja_muestra.to_excel("/content/datos_muestra_2000.xlsx", index=False, engine="openpyxl")

# Guardar en un único archivo CSV todo el dataset
hoja_madre.to_csv("/content/datos_limpios_con_temp_completo.csv", index=False)

# Guardar en un único archivo Excel dividiendo en hojas si es necesario
max_filas = 1000000
with pd.ExcelWriter("/content/datos_limpios_con_temp.xlsx", engine="openpyxl") as writer:
    for i in range(0, len(hoja_madre), max_filas):
        hoja_madre.iloc[i:i+max_filas].to_excel(writer, sheet_name=f"Parte_{i//max_filas+1}", index=False)

# Mostrar las primeras filas de la muestra
print("Ejemplo de datos de la muestra aleatoria de 2000 observaciones:")
print(hoja_muestra.head())